### Error_analysis
- The meain purpose of this document is to analyse the errors generated by the LLM Answers in order to improve our prompting

##### import error logs

In [1]:
import os 
os.chdir('../../airflow/dags/error_logs')
os.getcwd()

'/home/ysidhom/Documents/5IF/adhd_project/mental_health_disorders_analysis/airflow/dags/error_logs'

In [2]:
import pandas as pd
import numpy as np
error_df = None
for folder in os.listdir():
   for file in os.listdir(folder):
      if file.endswith('.csv'):
        path=os.path.join(folder, file)
        inter_df = pd.read_csv(path)
        error_df = pd.concat([error_df, inter_df], axis=0)

In [3]:
#Avoid analysis of duplicate rows
print(error_df.shape)
error_df=error_df.drop_duplicates()
print(error_df.shape)

(407, 4)
(92, 4)


In [4]:
list_er=['i8yyx3','1dpg746','1erdhk9']
test_err_df=error_df[error_df['id'].isin(list_er)]
print(test_err_df.shape)

(5, 4)


In [5]:
test=test_err_df['text'].iloc[0]
print(test)

[{'generated_text': '\n        Analyze the following Reddit post and provide concise answers to these features. \n        Use "Yes" or "No" for binary questions, and specify "Null" if information is unclear or not mentioned. Avoid explanations.\n\n        Features to extract:\n\n        Sentiment: positive/negative/neutral (if unclear put null)\n        Topic: Technology/medication/education/social \n        Personal Experience Shared: Yes/No\n        Mention of Solutions: Yes/No (if any solutions, advice, or recommendations are discussed just answer no explanation)\n        Gender of the Author: Male/Female/Null\n        Self-Diagnosis: Yes/No (keyword \'self-diagnosis\')\n        Self-Medication: Yes/No (keyword \'self-medication\' exits in this text ?)\n        Post for analysis:\n    \n        RSD and obsessive thinking doesnt help either.\ni said something that ive worded terribly on reddit earlier today and got absolutely shat on, i literally got compared to hitler.\n\nfor the pa

In [28]:
import ast
input_str = test
# Convert the string into a Python object (list of dictionaries)
parsed_data = ast.literal_eval(input_str)
generated_text = parsed_data[0]['generated_text']
generated_text_splitten = list(filter(None, generated_text_splitten))
# Remove strings that are only whitespace characters
generated_text_splitten = [line for line in generated_text_splitten if not line.isspace()]
generated_text_splitten = generated_text.split('\n\n')[-1].split('\n')
print('generated_text_splitten',generated_text_splitten)
# Remove leading and trailing spaces from each element
generated_text_splitten = [line.strip() for line in generated_text_splitten]

if(test_results('1',generated_text_splitten)):
    print('True')
    #continue
else:
    #refix the generated text
    print('False')
    flas_text=generated_text.split('\n\n')[-2].split('\n')
    flas_text = [line.strip() for line in flas_text]
    print(flas_text)


generated_text_splitten ['        The post expresses negative emotions about a personal experience with social interactions on Reddit. The author does not mention any solutions, and their gender is not specified. There is no self-diagnosis or explicit mention of self-medication in the post.']
Error_id: 1
['The post expresses negative emotions about a personal experience with social interactions on Reddit. The author does not mention any solutions, and their gender is not specified. There is no self-diagnosis or explicit mention of self-medication in the post.']
False
['Sentiment: Negative', 'Topic: Social', 'Personal Experience Shared: Yes', 'Mention of Solutions: Null', 'Gender of the Author: Null (No gender is mentioned)', 'Self-Diagnosis: Null', 'Self-Medication: Null']


In [26]:
print(generated_text.split('\n\n')[-2].split('\n'))

['        Sentiment: Negative', '        Topic: Social', '        Personal Experience Shared: Yes', '        Mention of Solutions: Null', '        Gender of the Author: Null (No gender is mentioned)', '        Self-Diagnosis: Null', '        Self-Medication: Null']


In [8]:
import ast  # To safely evaluate the string as a Python object
def fix_column_text(text):
   # Your input string
    input_str = text
    # Convert the string into a Python object (list of dictionaries)
    parsed_data = ast.literal_eval(input_str)
    generated_text = parsed_data[0]['generated_text']
    generated_text_splitten = generated_text.split('\n\n')[-1].split('\n')[-7:]
    # Remove leading and trailing spaces from each element
    generated_text_splitten = [line.strip() for line in generated_text_splitten]
    return generated_text_splitten




In [9]:
fix_column_text(test)

['The post expresses negative emotions about a personal experience with social interactions on Reddit. The author does not mention any solutions, and their gender is not specified. There is no self-diagnosis or explicit mention of self-medication in the post.']

In [69]:
error_df['text'] = error_df['text'].apply(fix_column_text)
print(error_df['text'].head(5))


0    [Sentiment: Neutral, Topic: Technology/Null, P...
1    [Sentiment: Neutral, Topic: Technology/medicat...
2    [The post expresses negative emotions about a ...
3    [Sentiment: positive, Topic: Technology/medica...
4    [Sentiment: Negative, Topic: Technology/medica...
Name: text, dtype: object


In [18]:
def test_results(id,text):
    try:
        response=text
        extracted_features = {
    "Sentiment": response[0].split(':')[1].strip().split(' ')[0],
    "Topic": response[1].split(':')[1].strip().split(' ')[0],
    "Personal_Experience": response[2].split(':')[1].strip().split(' ')[0],
    "Mention of Solutions": response[3].split(':')[1].strip().split(' ')[0],
    "Gender": response[4].split(':')[1].strip().split(' ')[0],
    "Self-Diagnosis": response[5].split(':')[1].strip().split(' ')[0],
    "Self-Medication": response[6].split(':')[1].strip().split(' ')[0]
    }
        return extracted_features
    except:
            
        return None

In [90]:
for i in range(len(error_df)):
    inter=error_df.iloc[i]
    test_results(inter['id'],inter['text'])
    

Error_id: i8yyx3
['The post expresses negative emotions about a personal experience with social interactions on Reddit. The author does not mention any solutions, and their gender is not specified. There is no self-diagnosis or explicit mention of self-medication in the post.']
Error_id: 1dpg746
['(The text seems to lack any clear discussion of technology, medication, education, or social topics, and the gender of the author is not specified. The author mentions personal experiences, solutions, self-diagnosis, but not self-medication.)']
Error_id: 1erdhk9
['I took the liberty to replace "Adderall" with "Technology" in the Topic section, as the post seems more about the experience of using Adderall (Technology) rather than discussing or debating about the topic itself.']
Error_id: 13kakv6
['This Reddit post describes a personal experience with someone who seems to struggle with focusing on tasks. The author is empathetic and understanding towards this person, acknowledging their intelli

In [67]:
text=error_df['text'][0]

# Your input string
input_str = text
# Convert the string into a Python object (list of dictionaries)
parsed_data = ast.literal_eval(input_str)
generated_text = parsed_data[0]['generated_text']
generated_text_splitten = generated_text.split('\n\n')[-1].split('\n')[-7:]
# Remove leading and trailing spaces from each element
generated_text_splitten = [line.strip() for line in generated_text_splitten]

print('Cleaned generated_text_splitten:', generated_text_splitten)



Cleaned generated_text_splitten: ['Sentiment: Neutral', 'Topic: Technology/Null', 'Personal Experience Shared: No', 'Mention of Solutions: No', 'Gender of the Author: Null', 'Self-Diagnosis: Null', 'Self-Medication: Null']


In [ ]:
def augmented_json_data(response):
    response=response[0]['generated_text'].split('\n\n')[-1].split('\n')[-7:]
    if response[0].startswith('--'):
        print(response[0])
        response=response[1:]
    
    # for el in response:
    #     print(el)
    #     print('\n')
    extracted_features = {
    "Sentiment": response[0].split(':')[1].strip().split(' ')[0],
    "Topic": response[1].split(':')[1].strip().split(' ')[0],
    "Personal_Experience": response[2].split(':')[1].strip().split(' ')[0],
    "Mention of Solutions": response[3].split(':')[1].strip().split(' ')[0],
    "Gender": response[4].split(':')[1].strip().split(' ')[0],
    "Self-Diagnosis": response[5].split(':')[1].strip().split(' ')[0],
    "Self-Medication": response[6].split(':')[1].strip().split(' ')[0]
    }

    return extracted_features
    